# Reading medical data

In [1]:
# importing library for pubmed article 
from lxml import etree

In [2]:
# Parsing the xml file
doc = etree.parse('../pubmed_result.xml')

In [3]:
root = doc.getroot()

In [4]:
# Tag name of the root node
root.tag

'PubmedArticleSet'

In [5]:
# Fetching PMID value 
for child in root.iter('PMID'):
    print( child.text)
    pmid = child.text

27246822


In [6]:
# Fetching Article Title
for child in root.iter('ArticleTitle'):
    print( child.text)
    title = child.text

Dabrafenib plus Trametinib: a Review in Advanced Melanoma with a BRAF (V600) Mutation.


In [7]:
# Fetching Abstract Text
for child in root.iter('AbstractText'):
    print( child.text)
    annotatedAbstract = child.text

The BRAF inhibitor dabrafenib (Tafinlar(®)) and the MEK inhibitor trametinib (Mekinist(®)) are indicated, as monotherapy or in combination with each other, for the treatment of patients with unresectable or metastatic melanoma with a BRAF (V600) mutation. This article reviews the therapeutic efficacy and tolerability of combination treatment with dabrafenib and trametinib in this indication and summarizes relevant pharmacological data. Dabrafenib plus trametinib significantly prolonged progression-free survival (PFS) and overall survival (OS), improved objective response rates (ORRs) and preserved health-related quality of life (HR-QOL) to a greater extent than dabrafenib (in the double-blind COMBI-d study) and vemurafenib (in the open-label COMBI-v study) in two large, randomized, phase III studies in treatment-naïve patients with unresectable or metastatic melanoma with BRAF (V600E/K) mutation. Limited treatment benefit with the combination was also seen in patients who had progresse

In [8]:
# Abstract Data

annotatedAbstract = annotatedAbstract.encode(encoding='UTF-8',errors='strict')

# Connecting to Grakn


In [9]:
from grakn.client import GraknClient 

In [10]:
# Inserting 

## Connecting and creating a client
with GraknClient(uri="localhost:48555") as client:
    
    ## Creating a session
    with client.session(keyspace="text") as session:
        ## creating a write transaction
        with session.transaction().write() as write_transaction:
            
            ## write transaction is open
            insert_iterator = write_transaction.query('insert $x isa pubmed-article, has pmid "{}",has title "{}" ,has annotated-abstract "{}" ;'.format(pmid,title,annotatedAbstract))           
    
            insert_list = insert_iterator.collect_concepts()
            print("Inserted pmid into the text_mining keyspace with ID: {0}".format(insert_list[0].id))
             
                     
            ## write transaction must always be committed (closed)
            write_transaction.commit()

Inserted pmid into the text_mining keyspace with ID: V4200


In [11]:
# Reading data form Grakn for text mining
client= GraknClient(uri = 'localhost:48555')
session = client.session(keyspace= 'text')
transaction = session.transaction().read()
query_iterator = transaction.query('match $a isa annotated-abstract;get;')
#(query='match $p isa pubmed-article;get;')





query_concepts = query_iterator.collect_concepts()

In [12]:
for q in query_concepts:
    print(q.id,"--",q.value())

V4240 -- b'The BRAF inhibitor dabrafenib (Tafinlar(\xc2\xae)) and the MEK inhibitor trametinib (Mekinist(\xc2\xae)) are indicated, as monotherapy or in combination with each other, for the treatment of patients with unresectable or metastatic melanoma with a BRAF (V600) mutation. This article reviews the therapeutic efficacy and tolerability of combination treatment with dabrafenib and trametinib in this indication and summarizes relevant pharmacological data. Dabrafenib plus trametinib significantly prolonged progression-free survival (PFS) and overall survival (OS), improved objective response rates (ORRs) and preserved health-related quality of life (HR-QOL) to a greater extent than dabrafenib (in the double-blind COMBI-d study) and vemurafenib (in the open-label COMBI-v study) in two large, randomized, phase III studies in treatment-na\xc3\xafve patients with unresectable or metastatic melanoma with BRAF (V600E/K) mutation. Limited treatment benefit with the combination was also se

In [13]:
# abstract id

abstract_id = query_concepts[0].id

In [14]:
abstract_id

'V4240'

In [15]:
# abstract value

abstract_value = query_concepts[0].value()

In [16]:
abstract_value

"b'The BRAF inhibitor dabrafenib (Tafinlar(\\xc2\\xae)) and the MEK inhibitor trametinib (Mekinist(\\xc2\\xae)) are indicated, as monotherapy or in combination with each other, for the treatment of patients with unresectable or metastatic melanoma with a BRAF (V600) mutation. This article reviews the therapeutic efficacy and tolerability of combination treatment with dabrafenib and trametinib in this indication and summarizes relevant pharmacological data. Dabrafenib plus trametinib significantly prolonged progression-free survival (PFS) and overall survival (OS), improved objective response rates (ORRs) and preserved health-related quality of life (HR-QOL) to a greater extent than dabrafenib (in the double-blind COMBI-d study) and vemurafenib (in the open-label COMBI-v study) in two large, randomized, phase III studies in treatment-na\\xc3\\xafve patients with unresectable or metastatic melanoma with BRAF (V600E/K) mutation. Limited treatment benefit with the combination was also seen

# Spacy -- Work

In [17]:
# import spacy 
import spacy

In [18]:
# Load custom trained model

nlp = spacy.load("/home/ubuntu//Grakn/Spacy Play/spacy_en_core_web_lg")

In [19]:
# Store pubmed-article fetched from grakn
# annotated_abstract

In [20]:
doc = nlp(abstract_value)

In [21]:
# Migrate Sentences

for sentence in doc.sents:
    print(sentence.text, sentence.sentiment,"\n","------" )

b'The BRAF inhibitor dabrafenib (Tafinlar(\xc2\xae)) and the MEK inhibitor trametinib (Mekinist(\xc2\xae)) are indicated, as monotherapy or in combination with each other, for the treatment of patients with unresectable or metastatic melanoma with a BRAF (V600) mutation. 0.0 
 ------
This article reviews the therapeutic efficacy and tolerability of combination treatment with dabrafenib and trametinib in this indication and summarizes relevant pharmacological data. 0.0 
 ------
Dabrafenib plus trametinib significantly prolonged progression-free survival (PFS) and overall survival (OS), improved objective response rates (ORRs) and preserved health-related quality of life (HR-QOL) to a greater extent than dabrafenib (in the double-blind COMBI-d study) and vemurafenib (in the open-label COMBI-v study) in two large, randomized, phase III studies in treatment- 0.0 
 ------
na\xc3\xafve patients with unresectable or metastatic melanoma with BRAF (V600E/K) mutation. 0.0 
 ------
Limited treatm

In [22]:
# Inserting 


for sentence in doc.sents:
    ## Connecting and creating a client
    with GraknClient(uri="localhost:48555") as client:

        ## Creating a session
        with client.session(keyspace="text") as session:
            ## creating a write transaction
            with session.transaction().write() as write_transaction:

                ## write transaction is open
                insert_iterator = write_transaction.query('insert $x isa sentence, has text "{}",has sentiment "{}" ;'.format(sentence.text,sentence.sentiment))           

                insert_list = insert_iterator.collect_concepts()
                print("Inserted sentence into the text keyspace with ID: {0}".format(insert_list[0].id))


                # Another insert for relation with abstract and its id
                insert_query = "match $s id {};$a id {};\
                insert $acs(contained-sentence: $s, abstract-container: $a) isa abstract-containing-sentence;".format(insert_list[0].id,abstract_id)
                
                insert_query_iterator = write_transaction.query(insert_query)
                
                insert_query_list = insert_query_iterator.collect_concepts()
                print("Inserted sentence attribute into the text keyspace with ID: {0}".format(insert_query_list[0].id))



                ## write transaction must always be committed (closed)
                write_transaction.commit()
    print("Insertion Done")

Inserted sentence into the text keyspace with ID: V40964136
Inserted sentence attribute into the text keyspace with ID: V4240
Insertion Done
Inserted sentence into the text keyspace with ID: V40968232
Inserted sentence attribute into the text keyspace with ID: V4240
Insertion Done
Inserted sentence into the text keyspace with ID: V8336
Inserted sentence attribute into the text keyspace with ID: V4240
Insertion Done
Inserted sentence into the text keyspace with ID: V12392
Inserted sentence attribute into the text keyspace with ID: V12392
Insertion Done
Inserted sentence into the text keyspace with ID: V40968312
Inserted sentence attribute into the text keyspace with ID: V40968312
Insertion Done
Inserted sentence into the text keyspace with ID: V40980520
Inserted sentence attribute into the text keyspace with ID: V4240
Insertion Done
Inserted sentence into the text keyspace with ID: V40992808
Inserted sentence attribute into the text keyspace with ID: V4240
Insertion Done
Inserted senten

In [23]:
# # Check the insertion id and value
# for i in insert_query_list:
#     print(i.id,i.value())

In [24]:
# # Quering ? reading all sentences

# with GraknClient(uri='localhost:48555') as client:
#     with client.session(keyspace="text_mining") as session:
#         # creating a query transaction
#         with session.transaction().read() as read_transaction:
            
#             read_iterator = read_transaction.query('match $s isa sentence;get;')
#             read_list = read_iterator.collect_concepts()
            
# #             for q in read_list:
# #                 print(q.id, q.is_type())
                
#         with session.transaction().write() as write_transaction:
            
#             for q in read_list:
#                 insert_query = "match $s id {};$a id {};\
#                 insert $acs(contained-sentence: $s, abstract-container: $a) isa abstract-containing-sentence;".format(q.id,abstract_id)


#              ## write transaction is open
#                 insert_iterator = write_transaction.query(insert_query)           

#                 insert_list = insert_iterator.collect_concepts()
#                 print("Inserted sentence attribute into the text_mining keyspace with ID: {0}".format(insert_list[0].id))
                
#                 ## write transaction must always be committed (closed)
#             write_transaction.commit()

            
            
            
        

# Tokens Insertion

## Token Extraction - Spacy

In [25]:
# spacy output of document 
doc

b'The BRAF inhibitor dabrafenib (Tafinlar(\xc2\xae)) and the MEK inhibitor trametinib (Mekinist(\xc2\xae)) are indicated, as monotherapy or in combination with each other, for the treatment of patients with unresectable or metastatic melanoma with a BRAF (V600) mutation. This article reviews the therapeutic efficacy and tolerability of combination treatment with dabrafenib and trametinib in this indication and summarizes relevant pharmacological data. Dabrafenib plus trametinib significantly prolonged progression-free survival (PFS) and overall survival (OS), improved objective response rates (ORRs) and preserved health-related quality of life (HR-QOL) to a greater extent than dabrafenib (in the double-blind COMBI-d study) and vemurafenib (in the open-label COMBI-v study) in two large, randomized, phase III studies in treatment-na\xc3\xafve patients with unresectable or metastatic melanoma with BRAF (V600E/K) mutation. Limited treatment benefit with the combination was also seen in pat

In [26]:
for token in doc:
    print(token.text,\
          token.lemma_,\
          token.pos_,\
          token.tag_,\
          token.dep_,\
          token.shape_,\
          token.is_alpha,\
          token.is_stop,
         token.ent_type_)

b'The b'the ADJ JJ compound x'Xxx False False 
BRAF BRAF PROPN NNP compound XXXX True False gene
inhibitor inhibitor NOUN NN compound xxxx True False 
dabrafenib dabrafenib NOUN NN nsubjpass xxxx True False drug
( ( PUNCT -LRB- punct ( False False 
Tafinlar(\xc2\xae tafinlar(\xc2\xae ADV RB appos Xxxxx(\xxd\xxx False False 
) ) PUNCT -RRB- punct ) False False 
) ) PUNCT -RRB- punct ) False False 
and and CCONJ CC cc xxx True True 
the the DET DT det xxx True True 
MEK MEK PROPN NNP compound XXX True False protien
inhibitor inhibitor NOUN NN compound xxxx True False 
trametinib trametinib NOUN NN conj xxxx True False drug
( ( PUNCT -LRB- punct ( False False 
Mekinist(\xc2\xae mekinist(\xc2\xae NOUN NN advmod Xxxxx(\xxd\xxx False False 
) ) PUNCT -RRB- punct ) False False 
) ) PUNCT -RRB- punct ) False False 
are be VERB VBP auxpass xxx True True 
indicated indicate VERB VBN ROOT xxxx True False 
, , PUNCT , punct , False False 
as as ADP IN advmod xx True True 
monotherapy monotherapy A

toxicity toxicity NOUN NN dobj xxxx True False 
relative relative ADJ JJ amod xxxx True False 
to to ADP IN prep xx True True 
dabrafenib dabrafenib NOUN NN pobj xxxx True False 
or or CCONJ CC cc xx True True 
vemurafenib vemurafenib NOUN NN conj xxxx True False drug
monotherapy monotherapy NOUN NN advmod xxxx True False 
, , PUNCT , punct , False False 
with with ADP IN prep xxxx True True 
most most ADJ JJS amod xxxx True True 
adverse adverse ADJ JJ amod xxxx True False 
events event NOUN NNS pobj xxxx True False 
( ( PUNCT -LRB- punct ( False False 
AEs AEs PROPN NNP appos XXx True False 
) ) PUNCT -RRB- punct ) False False 
mild mild ADJ JJ amod xxxx True False 
or or CCONJ CC cc xx True True 
moderate moderate ADJ JJ conj xxxx True False 
in in ADP IN prep xx True True 
severity severity NOUN NN pobj xxxx True False 
and and CCONJ CC cc xxx True True 
generally generally ADV RB advmod xxxx True False 
manageable manageable ADJ JJ conj xxxx True False 
. . PUNCT . punct . False F

### Quering Grakn DB for sentences

### Inserting token form queried sentences 

In [27]:
# Transaction query on doc

client = GraknClient(uri='localhost:48555')
session = client.session(keyspace="text")

# creating a query transaction
query_transaction = session.transaction() 

query_iterator = query_transaction.read().query('match $s isa sentence,has text $t;get $s;')

query_list = query_iterator.collect_concepts()

print(query_iterator)


# Write transaction

for q in query_list:
    print(q.id)
    
    
    for text in q.attributes():
        #esult.append(text.value())
        if text.type().label() == 'text':
            print(text.value())
    
            sent_id = q.id
            # Tokenizing extracted sentence
            doc = nlp(text.value())
            for token in doc:
                token_text = token.text
                token_lemma = token.lemma_
                token_pos = token.pos_
                token_tag = token.tag_
                token_ner = token.ent_type_


                ## Creating a session
                with client.session(keyspace="text") as session:
                    ## creating a write transaction
                    with session.transaction().write() as write_transaction:
                        # Grakn Insert Query
                        insert_query = 'match $s id {};\
                        insert $t isa token,\
                        has tag "{}",\
                        has lemma "{}",\
                        has ner "{}";\
                        $sct(sentence-container: $s,contained-token : $t) isa sentence-containing-token;'.format(sent_id,token_tag,token_lemma,token_ner)

                        #print(insert_query,"\n")

                        insert_query_iterator = write_transaction.query(insert_query)

                        insert_query_list = insert_query_iterator.collect_concepts()
                        print("Inserted token attribute into the text keyspace with ID: {0}".format(insert_query_list[0].id))

                        ## write transaction must always be committed (closed)
                        write_transaction.commit()

        
print('write completed')
# query_transaction.read().close()
# session.close()
# client.close()
                
             

V8336
Dabrafenib plus trametinib significantly prolonged progression-free survival (PFS) and overall survival (OS), improved objective response rates (ORRs) and preserved health-related quality of life (HR-QOL) to a greater extent than dabrafenib (in the double-blind COMBI-d study) and vemurafenib (in the open-label COMBI-v study) in two large, randomized, phase III studies in treatment-
Inserted token attribute into the text keyspace with ID: V8336
Inserted token attribute into the text keyspace with ID: V20584
Inserted token attribute into the text keyspace with ID: V20568
Inserted token attribute into the text keyspace with ID: V8336
Inserted token attribute into the text keyspace with ID: V8336
Inserted token attribute into the text keyspace with ID: V8336
Inserted token attribute into the text keyspace with ID: V32856
Inserted token attribute into the text keyspace with ID: V8336
Inserted token attribute into the text keyspace with ID: V57488
Inserted token attribute into the text

Inserted token attribute into the text keyspace with ID: V12400
Inserted token attribute into the text keyspace with ID: V323696
V40964136
b'The BRAF inhibitor dabrafenib (Tafinlar(\xc2\xae)) and the MEK inhibitor trametinib (Mekinist(\xc2\xae)) are indicated, as monotherapy or in combination with each other, for the treatment of patients with unresectable or metastatic melanoma with a BRAF (V600) mutation.
Inserted token attribute into the text keyspace with ID: V164056
Inserted token attribute into the text keyspace with ID: V327792
Inserted token attribute into the text keyspace with ID: V40964136
Inserted token attribute into the text keyspace with ID: V335984
Inserted token attribute into the text keyspace with ID: V40964136
Inserted token attribute into the text keyspace with ID: V176344
Inserted token attribute into the text keyspace with ID: V135264
Inserted token attribute into the text keyspace with ID: V40964136
Inserted token attribute into the text keyspace with ID: V40964

Inserted token attribute into the text keyspace with ID: V40968312
Inserted token attribute into the text keyspace with ID: V245848
Inserted token attribute into the text keyspace with ID: V40968312
Inserted token attribute into the text keyspace with ID: V557168
Inserted token attribute into the text keyspace with ID: V307304
V40980520
Combination therapy did not increase overall toxicity relative to dabrafenib or vemurafenib monotherapy, with most adverse events (AEs) mild or moderate in severity and generally manageable.
Inserted token attribute into the text keyspace with ID: V41213992
Inserted token attribute into the text keyspace with ID: V41218088
Inserted token attribute into the text keyspace with ID: V41222184
Inserted token attribute into the text keyspace with ID: V40980520
Inserted token attribute into the text keyspace with ID: V40980520
Inserted token attribute into the text keyspace with ID: V319592
Inserted token attribute into the text keyspace with ID: V40980520
Ins

# Migrate Mined relation 

In [54]:
# new_query = "match $s isa sentence, has text 'Thus, dabrafenib plus trametinib provides an important treatment option for patients with BRAF (V600) mutation-positive unresectable or metastatic melanoma.''; \
# $subjectToken isa token, has lemma 'melanoma';\
# $objectToken1 isa token, has lemma 'trametinib';\
# $objectToken2 isa token, has lemma 'dabrafenib';\
# insert $r (mentioning-sentence: $s, subject: $subjectToken, object: $objectToken1, object: $objectToken2) isa mined-relation, has confidence 1.0, has relation-type 'treatment';"

In [51]:
new_query = "match $s id V12400;\
            $subjectToken id V315504;\
            $objectToken1 id V131160;\
            $objectToken2 id V258192;\
            insert $r (mentioning-sentence: $s, subject: $subjectToken, object: $objectToken1, object: $objectToken2) isa mined-relation, has confidence 1.0, has relation-type 'treatment';"
            


In [52]:
new_query

"match $s id V12400;            $subjectToken id V315504;            $objectToken1 id V131160;            $objectToken2 id V258192;            insert $r (mentioning-sentence: $s, subject: $subjectToken, object: $objectToken1, object: $objectToken2) isa mined-relation, has confidence 1.0, has relation-type 'treatment';"

In [53]:
# # Migrating Mined Relation inot Grakn
# # Using Dummy Relationship

with GraknClient(uri='localhost:48555') as client:

    with client.session(keyspace="text") as session:
            
        with session.transaction().write() as write_transaction:

             ## write transaction is open
                   
                insert_query = "match $s isa sentence, has text 'Thus, dabrafenib plus trametinib provides an important treatment option for patients with BRAF (V600) mutation-positive unresectable or metastatic melanoma.'; " +\
                "$subjectToken isa token, has lemma 'melanoma', has start-position 1720.0, has end-position 1728.0; " +\
                "$objectToken1 isa token, has lemma 'trametinib', has start-position 1595.0, has end-position 1605.0; " +\
                "$objectToken2 isa token, has lemma 'dabrafenib', has start-position 1579.0, has end-position 1589.0; " +\
                "insert $r (mentioning-sentence: $s, subject: $subjectToken, object: $objectToken1, object: $objectToken2) isa mined-relation, has confidence 1.0, has relation-type 'treatment';"
                
                
                #print(insert_query,"\n")

                insert_query_iterator = write_transaction.query(new_query)#(insert_query)

                insert_query_list = insert_query_iterator.collect_concepts()
                print(insert_query_list)
                #("Inserted relationship attribute into the text keyspace with ID: {0}".format(insert_query_list[0].id))

                ## write transaction must always be committed (closed)
                write_transaction.commit()
                
                
                print("Inserted mined-relations")


            
            
            
        

[<grakn.service.Session.Concept.Concept.Entity object at 0x7f5acbd695c0>, <grakn.service.Session.Concept.Concept.Relation object at 0x7f5acbd696d8>, <grakn.service.Session.Concept.Concept.Entity object at 0x7f5acbd6ce80>, <grakn.service.Session.Concept.Concept.Entity object at 0x7f5acbd6c9b0>, <grakn.service.Session.Concept.Concept.Entity object at 0x7f5acbd6cda0>]
Inserted mined-relations


In [55]:
insert_query_list

# Migrate Entity Mention and Mention 

In [29]:
for token in doc:
    print(token.text,"-->",
         token.ent_type_
        )

Fewer --> 
skin --> 
- --> 
related --> 
AEs --> 
( --> 
e.g. --> 
cutaneous --> 
malignancies --> 
, --> 
hyperkeratinosis --> 
and --> 
hand --> 
- --> 
foot --> 
syndrome --> 
) --> 
were --> 
reported --> 
with --> 
combination --> 
therapy --> 
than --> 
with --> 
dabrafenib --> 
or --> 
vemurafenib --> drug
, --> 
probably --> 
because --> 
of --> 
reduced --> 
paradoxical --> 
activation --> 
of --> 
the --> 
MAPK --> 
pathway --> 
. --> 


In [30]:
# Transaction query on doc

with GraknClient(uri='localhost:48555') as client:

    with client.session(keyspace="text") as session:
            
        with session.transaction().read() as read_transaction:
            read_query_iterator = read_transaction.query('match $s isa sentence,has text $t;get $s;')
            read_query_list = read_query_iterator.collect_concepts()
            
            print(query_iterator)


            # Write transaction

            for q in read_query_list:
                print(q.id)
    
                # Inner For loop for reading the text from the retrieved sentence concpet
                for text in q.attributes():
                    #esult.append(text.value())
                    if text.type().label() == 'text':
                        print(text.value())
    
                        sent_id = q.id
                        # Tokenizing extracted sentence
                        doc = nlp(text.value())
                        for token in doc:
                            if token.ent_type_:
                                entity_value = token.text
                            
                                entity_type = token.ent_type_
                                print(entity_value, "-->",entity_type)


                            ## Creating a session
                            with client.session(keyspace="text") as session:
                                ## creating a write transaction
                                with session.transaction().write() as write_transaction:
                                    # Grakn Insert Query
                                    insert_query = 'match $s id {};\
                                    insert $e isa entity-mention,\
                                    has value "{}",\
                                    has entity-type "{}";\
                                    $m(mentioned-entity: $e,mentioning-sentence : $s) isa mention;'.format(sent_id,entity_value,entity_type)

                                    #print(insert_query,"\n")

                                    insert_query_iterator = write_transaction.query(insert_query)

                                    insert_query_list = insert_query_iterator.collect_concepts()
                                    #print("Inserted token attribute into the text keyspace with ID: {0}".format(insert_query_list[0].id))

                                    ## write transaction must always be committed (closed)
                                    write_transaction.commit()
                        print('Entity Mention Insertion completed')

                
             

V8336
Dabrafenib plus trametinib significantly prolonged progression-free survival (PFS) and overall survival (OS), improved objective response rates (ORRs) and preserved health-related quality of life (HR-QOL) to a greater extent than dabrafenib (in the double-blind COMBI-d study) and vemurafenib (in the open-label COMBI-v study) in two large, randomized, phase III studies in treatment-
Dabrafenib --> drug
trametinib --> drug
Entity Mention Insertion completed
V12392
na\xc3\xafve patients with unresectable or metastatic melanoma with BRAF (V600E/K) mutation.
BRAF --> gene
V600E --> varient
/ --> varient
K --> varient
Entity Mention Insertion completed
V12400
Thus, dabrafenib plus trametinib provides an important treatment option for patients with BRAF (V600) mutation-positive unresectable or metastatic melanoma.'
trametinib --> drug
BRAF --> gene
V600 --> varient
melanoma --> diesease
Entity Mention Insertion completed
V40964136
b'The BRAF inhibitor dabrafenib (Tafinlar(\xc2\xae)) and

# Migrate entity-containing-token 

In [92]:
# Transaction query on doc

with GraknClient(uri='localhost:48555') as client:

    with client.session(keyspace="text") as session:
            
        with session.transaction().read() as read_transaction:
            read_query_iterator = read_transaction.query('match $e isa entity-mention;get $e;')
            read_query_list = read_query_iterator.collect_concepts()
            
            for q in read_query_list:
                print(q.id)
                for value in q.attributes():
                    if value.type().label() == 'value':
                        #print(value.value())
                        
                        spacy_entity_doc = nlp(value.value())
                        for token in spacy_entity_doc:
                            print(token.text, "-->", token.lemma_)
                            
#                             with client.session(keyspace="text_mining") as session:
#                                 ## creating a write transaction
#                                 with session.transaction().write() as write_transaction:
#                                     # Grakn Insert Query
#                                     insert_query = 'match $e isa entity-mention,\
#                                     has value "{}";\
#                                     $t isa token,\
#                                     has lemma "{}";\
#                                     insert $ect(containing-entity: $e,contained-token : $t) isa entity-containing-token;'\
#                                     .format(token.text,token.lemma_)

#                                     #print(insert_query,"\n")

#                                     insert_query_iterator = write_transaction.query(insert_query)

#                                     insert_query_list = insert_query_iterator.collect_concepts()
#                                     #print("Inserted token attribute into the text_mining keyspace with ID: {0}".format(insert_query_list[0].id))

#                                     ## write transaction must always be committed (closed)
#                                     write_transaction.commit()
#                         print('Entity Containing Token Mention Insertion completed')
            

                
             

V303240
vemurafenib --> vemurafenib
V307336
vemurafenib --> vemurafenib
V311432
vemurafenib --> vemurafenib
V336008
dabrafenib --> dabrafenib
V340104
dabrafenib --> dabrafenib
V344312
vemurafenib --> vemurafenib
V348296
trametinib --> trametinib
V348408
vemurafenib --> vemurafenib
V352392
trametinib --> trametinib
V360584
trametinib --> trametinib
V360696
BRAF --> BRAF
V364792
MEK --> MEK
V372984
trametinib --> trametinib
V376968
V600 --> v600
V377080
trametinib --> trametinib
V381064
V600 --> v600
V381072
vemurafenib --> vemurafenib
V385160
V600 --> v600
V385272
melanoma --> melanoma
V389264
vemurafenib --> vemurafenib
V397448
vemurafenib --> vemurafenib
V401656
V600 --> v600
V405632
vemurafenib --> vemurafenib
V405648
trametinib --> trametinib
V405752
vemurafenib --> vemurafenib
V409728
trametinib --> trametinib
V409736
vemurafenib --> vemurafenib
V409744
trametinib --> trametinib
V409848
vemurafenib --> vemurafenib
V413824
trametinib --> trametinib
V413832
vemurafenib --> vemurafeni

trametinib --> trametinib
V41324576
V600 --> v600
V41328664
trametinib --> trametinib
V41336856
trametinib --> trametinib
V41345048
trametinib --> trametinib
V41345056
vemurafenib --> vemurafenib
V41361440
vemurafenib --> vemurafenib
V41377824
trametinib --> trametinib
V41381912
vemurafenib --> vemurafenib
V41381920
trametinib --> trametinib
V41394200
vemurafenib --> vemurafenib
V41394208
trametinib --> trametinib
V41398296
vemurafenib --> vemurafenib
V41398304
trametinib --> trametinib
V41402392
vemurafenib --> vemurafenib
V41402400
trametinib --> trametinib
V41406496
trametinib --> trametinib
V41410584
vemurafenib --> vemurafenib
V41418776
trametinib --> trametinib
V41422880
trametinib --> trametinib
V41426968
trametinib --> trametinib
V41435160
trametinib --> trametinib
V41439256
trametinib --> trametinib
V41439264
trametinib --> trametinib
V41443352
trametinib --> trametinib
V41443360
BRAF --> BRAF
V41447448
trametinib --> trametinib
V41451544
trametinib --> trametinib
V41451552
BR

# Close Grakn Connection

In [ ]:
client.close()